# Import Statement

In [1]:
from dask import dataframe as dd

# Basics

In [5]:
%%time
ddf = dd.read_csv('sample_data.csv')

CPU times: user 57.9 ms, sys: 75 ms, total: 133 ms
Wall time: 185 ms


## Bonus: Parquet

In [13]:
%%time
df = dd.read_parquet('sample_data_parquet')

CPU times: user 145 ms, sys: 135 ms, total: 280 ms
Wall time: 776 ms


In [6]:
ddf.head()

,Unnamed: 0,alphabet,useless_letter,Z,Y,X,W,V,U,T,...,J,I,H,G,F,E,D,C,B,A
0,0,ABCDEFGHIJKLMNOPQRSTUVWXYZ,A,1,66,69,4,70,64,83,...,36,28,33,57,23,86,48,30,91,84
1,1,ABCDEFGHIJKLMNOPQRSTUVWXYZ,F,25,26,36,34,0,75,60,...,42,91,8,24,64,13,43,47,94,11
2,2,ABCDEFGHIJKLMNOPQRSTUVWXYZ,H,40,67,36,54,46,5,57,...,82,76,24,60,3,55,64,28,26,89
3,3,ABCDEFGHIJKLMNOPQRSTUVWXYZ,F,91,98,8,36,17,3,29,...,88,24,7,51,52,87,1,6,19,48
4,4,ABCDEFGHIJKLMNOPQRSTUVWXYZ,C,87,68,73,78,39,67,57,...,9,14,0,2,51,18,95,71,28,13


In [9]:
type(ddf)

dask.dataframe.core.DataFrame

In [10]:
ddf

,Unnamed: 0,alphabet,useless_letter,Z,Y,X,W,V,U,T,S,R,Q,P,O,N,M,L,K,J,I,H,G,F,E,D,C,B,A
npartitions=16,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,object,object,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [14]:
ddf.npartitions

16

## Computation

In [12]:
%%time
ddf['mean'] = ddf.mean(axis=1)

CPU times: user 38.4 ms, sys: 23.8 ms, total: 62.2 ms
Wall time: 156 ms


# Best Practices

In general, full data shuffle is expensive. Avoid repetition of set_index and keep all sorting operations within partitions.

In [32]:
def computation_within_partitions(df):
    for col in df.columns()[3:]:
        df[f'{col}_STD'] = df[col].mean()
    return df

out_meta = ddf.dtypes.to_dict()

for col in ddf.columns.tolist()[3:]:
    out_meta.update({f"{col}_STD":float})

ddf = ddf.map_partitions(computation_within_partitions, meta=out_meta)

In [ ]:
%%time
df = ddf.compute()

In [ ]:
type(df)